## Part 1: CUDA C++ (Primary)

### GPU Check

In [ ]:
!nvcc --version
!nvidia-smi --query-gpu=name,memory.total --format=csv

### What is Shared Memory?

**Shared memory** is fast on-chip memory shared by all threads in a block.

```
┌─────────────────────────────────────────────────────────────┐
│                         GPU                                 │
├─────────────────────────────────────────────────────────────┤
│  ┌─────────────────┐  ┌─────────────────┐                   │
│  │     Block 0     │  │     Block 1     │  ...              │
│  │  ┌───────────┐  │  │  ┌───────────┐  │                   │
│  │  │  Shared   │  │  │  │  Shared   │  │  ← Fast! ~5 cycles│
│  │  │  Memory   │  │  │  │  Memory   │  │                   │
│  │  │ (48-164KB)│  │  │  │ (48-164KB)│  │                   │
│  │  └───────────┘  │  │  └───────────┘  │                   │
│  │   ↑ ↑ ↑ ↑ ↑ ↑   │  │   ↑ ↑ ↑ ↑ ↑ ↑   │                   │
│  │   T0 T1 T2...   │  │   T0 T1 T2...   │                   │
│  └─────────────────┘  └─────────────────┘                   │
├─────────────────────────────────────────────────────────────┤
│                    Global Memory                            │
│                    (VRAM, ~400 cycles)                      │
└─────────────────────────────────────────────────────────────┘
```

**Key Properties:**
- ~100x faster than global memory
- Shared only within a block (not across blocks)
- Limited size: 48-164 KB per SM
- Requires `__syncthreads()` for synchronization

In [ ]:
%%writefile shared_basics.cu
/**
 * Shared Memory Basics
 * 
 * Demonstrates static and dynamic shared memory allocation.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256

// Static shared memory: size known at compile time
__global__ void staticShared(float *input, float *output, int n) {
    __shared__ float cache[BLOCK_SIZE];  // Static allocation
    
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;
    
    // Load from global to shared
    if (idx < n) {
        cache[tid] = input[idx];
    }
    
    __syncthreads();  // Barrier: wait for all threads to finish loading
    
    // Now all threads can access any element in cache
    // Example: each thread reads its neighbor's value
    if (idx < n && tid > 0) {
        output[idx] = cache[tid] + cache[tid - 1];
    } else if (idx < n) {
        output[idx] = cache[tid];
    }
}

// Dynamic shared memory: size specified at launch
__global__ void dynamicShared(float *input, float *output, int n) {
    extern __shared__ float cache[];  // Size set at launch
    
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;
    
    if (idx < n) {
        cache[tid] = input[idx];
    }
    
    __syncthreads();
    
    if (idx < n && tid > 0) {
        output[idx] = cache[tid] + cache[tid - 1];
    } else if (idx < n) {
        output[idx] = cache[tid];
    }
}

int main() {
    printf("=== Shared Memory Basics ===\n\n");
    
    int n = 1024;
    size_t size = n * sizeof(float);
    
    float *h_input = (float*)malloc(size);
    float *h_output = (float*)malloc(size);
    
    for (int i = 0; i < n; i++) h_input[i] = i;
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    
    int blocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Test static shared memory
    printf("Static shared memory:\n");
    staticShared<<<blocks, BLOCK_SIZE>>>(d_input, d_output, n);
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);
    printf("  output[5] = %.0f + %.0f = %.0f ✓\n", 
           h_input[5], h_input[4], h_output[5]);
    
    // Test dynamic shared memory
    printf("\nDynamic shared memory:\n");
    size_t sharedMemSize = BLOCK_SIZE * sizeof(float);
    dynamicShared<<<blocks, BLOCK_SIZE, sharedMemSize>>>(d_input, d_output, n);
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);
    printf("  output[10] = %.0f + %.0f = %.0f ✓\n", 
           h_input[10], h_input[9], h_output[10]);
    
    // Query shared memory info
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("\nDevice shared memory per block: %zu KB\n", 
           prop.sharedMemPerBlock / 1024);
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 shared_basics.cu -o shared_basics && ./shared_basics

### The `__syncthreads()` Barrier

**Critical rule:** After loading data into shared memory, you MUST call `__syncthreads()` before any thread reads data written by another thread.

```cpp
// Thread 0 writes, Thread 1 reads
cache[threadIdx.x] = input[idx];  // Thread 0 writes cache[0]
__syncthreads();                  // REQUIRED!
float val = cache[threadIdx.x - 1];  // Thread 1 reads cache[0]
```

**Warning:** `__syncthreads()` in conditional code can cause deadlocks!

### Matrix Transpose with Shared Memory

Remember from Day 1: naive transpose has uncoalesced writes. 
Solution: use shared memory as a staging buffer!

In [ ]:
%%writefile transpose_shared.cu
/**
 * Matrix Transpose with Shared Memory
 * 
 * Uses shared memory to achieve coalesced reads AND writes.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8
#define WIDTH 4096
#define HEIGHT 4096
#define ITERATIONS 100

// Naive transpose (from Day 1)
__global__ void transposeNaive(float *out, float *in, int width, int height) {
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    if (x < width && y < height) {
        out[x * height + y] = in[y * width + x];  // Strided write!
    }
}

// Shared memory transpose
__global__ void transposeShared(float *out, float *in, int width, int height) {
    __shared__ float tile[TILE_DIM][TILE_DIM];
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // Coalesced read from global memory into shared memory
    if (x < width && y < height) {
        tile[threadIdx.y][threadIdx.x] = in[y * width + x];
    }
    
    __syncthreads();
    
    // Calculate new coordinates for transposed output
    x = blockIdx.y * TILE_DIM + threadIdx.x;  // Swap block indices
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    // Coalesced write from shared memory to global memory
    if (x < height && y < width) {
        out[y * height + x] = tile[threadIdx.x][threadIdx.y];  // Note: indices swapped!
    }
}

int main() {
    printf("=== Matrix Transpose: Naive vs Shared Memory ===\n");
    printf("Matrix: %d x %d\n\n", WIDTH, HEIGHT);
    
    size_t size = WIDTH * HEIGHT * sizeof(float);
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    
    // Initialize
    float *h_in = (float*)malloc(size);
    for (int i = 0; i < WIDTH * HEIGHT; i++) h_in[i] = i;
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);
    
    dim3 block(TILE_DIM, TILE_DIM);
    dim3 grid((WIDTH + TILE_DIM - 1) / TILE_DIM, (HEIGHT + TILE_DIM - 1) / TILE_DIM);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark naive
    transposeNaive<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        transposeNaive<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_time;
    cudaEventElapsedTime(&naive_time, start, stop);
    naive_time /= ITERATIONS;
    float naive_bw = 2.0f * size / (naive_time * 1e6);
    
    // Benchmark shared memory
    transposeShared<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        transposeShared<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float shared_time;
    cudaEventElapsedTime(&shared_time, start, stop);
    shared_time /= ITERATIONS;
    float shared_bw = 2.0f * size / (shared_time * 1e6);
    
    printf("❌ Naive Transpose:  %.3f ms, %.2f GB/s\n", naive_time, naive_bw);
    printf("✅ Shared Memory:    %.3f ms, %.2f GB/s\n", shared_time, shared_bw);
    printf("\nSpeedup: %.2fx\n", naive_time / shared_time);
    
    // Verify correctness
    float *h_out = (float*)malloc(size);
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
    
    bool correct = true;
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            if (h_out[j * HEIGHT + i] != h_in[i * WIDTH + j]) {
                correct = false;
            }
        }
    }
    printf("\nVerification: %s\n", correct ? "✅ PASSED" : "❌ FAILED");
    
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 transpose_shared.cu -o transpose_shared && ./transpose_shared

### Tiled Matrix Multiplication

Shared memory really shines for matrix multiplication. Each element of C requires an entire row of A and column of B. Without shared memory, we'd reload these elements repeatedly.

```
Without tiling: Each thread loads WIDTH elements from A and B
With tiling:    Threads cooperatively load TILE_SIZE elements, reuse them
```

In [ ]:
%%writefile matmul_tiled.cu
/**
 * Tiled Matrix Multiplication with Shared Memory
 * 
 * Demonstrates massive speedup through data reuse.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_SIZE 32
#define M 1024
#define N 1024
#define K 1024
#define ITERATIONS 10

// Naive matrix multiplication (no tiling)
__global__ void matmulNaive(float *C, float *A, float *B, int m, int n, int k) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < m && col < n) {
        float sum = 0.0f;
        for (int i = 0; i < k; i++) {
            sum += A[row * k + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

// Tiled matrix multiplication
__global__ void matmulTiled(float *C, float *A, float *B, int m, int n, int k) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];
    
    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;
    
    float sum = 0.0f;
    
    // Loop over tiles
    for (int t = 0; t < (k + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Collaboratively load tile into shared memory
        int tiledCol = t * TILE_SIZE + threadIdx.x;
        int tiledRow = t * TILE_SIZE + threadIdx.y;
        
        if (row < m && tiledCol < k)
            As[threadIdx.y][threadIdx.x] = A[row * k + tiledCol];
        else
            As[threadIdx.y][threadIdx.x] = 0.0f;
        
        if (tiledRow < k && col < n)
            Bs[threadIdx.y][threadIdx.x] = B[tiledRow * n + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0f;
        
        __syncthreads();
        
        // Compute partial dot product using shared memory
        for (int i = 0; i < TILE_SIZE; i++) {
            sum += As[threadIdx.y][i] * Bs[i][threadIdx.x];
        }
        
        __syncthreads();
    }
    
    if (row < m && col < n) {
        C[row * n + col] = sum;
    }
}

int main() {
    printf("=== Tiled Matrix Multiplication ===\n");
    printf("Matrices: A(%dx%d) x B(%dx%d) = C(%dx%d)\n\n", M, K, K, N, M, N);
    
    size_t sizeA = M * K * sizeof(float);
    size_t sizeB = K * N * sizeof(float);
    size_t sizeC = M * N * sizeof(float);
    
    float *h_A = (float*)malloc(sizeA);
    float *h_B = (float*)malloc(sizeB);
    float *h_C = (float*)malloc(sizeC);
    
    for (int i = 0; i < M * K; i++) h_A[i] = 1.0f;
    for (int i = 0; i < K * N; i++) h_B[i] = 1.0f;
    
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, sizeA);
    cudaMalloc(&d_B, sizeB);
    cudaMalloc(&d_C, sizeC);
    
    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);
    
    dim3 block(TILE_SIZE, TILE_SIZE);
    dim3 grid((N + TILE_SIZE - 1) / TILE_SIZE, (M + TILE_SIZE - 1) / TILE_SIZE);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark naive
    matmulNaive<<<grid, block>>>(d_C, d_A, d_B, M, N, K);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        matmulNaive<<<grid, block>>>(d_C, d_A, d_B, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_time;
    cudaEventElapsedTime(&naive_time, start, stop);
    naive_time /= ITERATIONS;
    float naive_gflops = (2.0f * M * N * K) / (naive_time * 1e6);
    
    // Benchmark tiled
    matmulTiled<<<grid, block>>>(d_C, d_A, d_B, M, N, K);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        matmulTiled<<<grid, block>>>(d_C, d_A, d_B, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float tiled_time;
    cudaEventElapsedTime(&tiled_time, start, stop);
    tiled_time /= ITERATIONS;
    float tiled_gflops = (2.0f * M * N * K) / (tiled_time * 1e6);
    
    printf("❌ Naive:  %.3f ms, %.2f GFLOPS\n", naive_time, naive_gflops);
    printf("✅ Tiled:  %.3f ms, %.2f GFLOPS\n", tiled_time, tiled_gflops);
    printf("\nSpeedup: %.2fx\n", naive_time / tiled_time);
    
    // Verify
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);
    bool correct = true;
    float expected = K;  // Each element should be K (sum of K 1.0s)
    for (int i = 0; i < 100; i++) {
        if (h_C[i] != expected) { correct = false; break; }
    }
    printf("\nVerification: %s (expected: %.0f, got: %.0f)\n", 
           correct ? "✅ PASSED" : "❌ FAILED", expected, h_C[0]);
    
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 matmul_tiled.cu -o matmul_tiled && ./matmul_tiled

### Shared Memory Summary

| Concept | Description |
|---------|-------------|
| `__shared__` | Declare shared memory variable |
| `extern __shared__` | Dynamic size (set at launch) |
| `__syncthreads()` | Barrier - all threads must reach |
| Scope | Per-block (not visible to other blocks) |
| Size | 48-164 KB per SM |

**When to use shared memory:**
1. Data reuse within a block (matrix multiply, convolutions)
2. Avoiding uncoalesced access (transpose)
3. Thread communication within a block
4. Reducing global memory bandwidth

---

## Part 2: Python/Numba (Optional)

In [ ]:
# Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

from numba import cuda
import numpy as np

print(f"CUDA Device: {cuda.get_current_device().name.decode()}")

In [ ]:
# Shared memory in Numba

TILE_SIZE = 32

@cuda.jit
def matmul_shared(C, A, B):
    """Tiled matrix multiplication with shared memory."""
    # Shared memory tiles
    sA = cuda.shared.array((TILE_SIZE, TILE_SIZE), dtype=np.float32)
    sB = cuda.shared.array((TILE_SIZE, TILE_SIZE), dtype=np.float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    row = cuda.blockIdx.y * TILE_SIZE + ty
    col = cuda.blockIdx.x * TILE_SIZE + tx
    
    m, k = A.shape
    _, n = B.shape
    
    result = 0.0
    
    for t in range((k + TILE_SIZE - 1) // TILE_SIZE):
        # Load tile into shared memory
        tiled_col = t * TILE_SIZE + tx
        tiled_row = t * TILE_SIZE + ty
        
        if row < m and tiled_col < k:
            sA[ty, tx] = A[row, tiled_col]
        else:
            sA[ty, tx] = 0.0
            
        if tiled_row < k and col < n:
            sB[ty, tx] = B[tiled_row, col]
        else:
            sB[ty, tx] = 0.0
        
        cuda.syncthreads()
        
        # Compute partial product
        for i in range(TILE_SIZE):
            result += sA[ty, i] * sB[i, tx]
        
        cuda.syncthreads()
    
    if row < m and col < n:
        C[row, col] = result

# Test
M, K, N = 512, 512, 512
A = np.ones((M, K), dtype=np.float32)
B = np.ones((K, N), dtype=np.float32)
C = np.zeros((M, N), dtype=np.float32)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

threads = (TILE_SIZE, TILE_SIZE)
blocks = ((N + TILE_SIZE - 1) // TILE_SIZE, (M + TILE_SIZE - 1) // TILE_SIZE)

matmul_shared[blocks, threads](d_C, d_A, d_B)
C = d_C.copy_to_host()

print(f"Result C[0,0] = {C[0,0]} (expected: {K})")
print(f"Verification: {'✅ PASSED' if C[0,0] == K else '❌ FAILED'}")